# Bert Named Entity Recognition for Cabinette
Adapted from [NielsRogge's custom Bert NER notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT.ipynb).

Any references to `foodner`, the original name of the project can be safely ignored.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
data = pd.read_csv("dataset/pytorch_dataset.csv", encoding='unicode_escape')

# Shuffling the data
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,sentence,word_labels
0,"10 minutes before the end of cooking, get a fe...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,Add onions and cook for 5 minutes,"O,U-FOOD,O,O,O,O,O"
2,"Add the meat back to the soup, setting some sl...","O,O,O,O,O,O,O,O,O,O,O,O,U-FOOD,O,O"
3,Add 4l of water to the tea concentrate and pou...,"O,O,O,U-FOOD,O,O,U-FOOD,O,O,O,O,O,O,O,O,O"
4,"Add burned black bread, a handful of raisins, ...","O,O,O,U-FOOD,O,O,O,O,U-FOOD,O,O,O,O,O,U-FOOD,O..."


These are the number of sentences we will train our model on.

In [4]:
data.sentence.count()

676

In [5]:
# TODO: We should really stop hardcoding these tags
tags=('O','B-FOOD','I-FOOD','L-FOOD','U-FOOD')
label2id = {k: v for v, k in enumerate(tags)}
id2label = {v: k for v, k in enumerate(tags)}
label2id

{'O': 0, 'B-FOOD': 1, 'I-FOOD': 2, 'L-FOOD': 3, 'U-FOOD': 4}

Let's verify that a random sentence and its corresponding tags are correct:

In [6]:
data.iloc[41].sentence

'Add and roast either diced or grated onion or crushed and sliced garlic until translucent.'

In [7]:
data.iloc[41].word_labels

'O,O,O,O,O,O,O,U-FOOD,O,O,O,O,U-FOOD,O,O,O'

In [8]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()
    
    # sanity check
    text_labels = str(text_labels)

    for word, label in zip(sentence.split(), text_labels.split(",")):
        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Note that this is a **design decision**. You could also decide to only label the first wordpiece of each word and let the model only learn this (this is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063)). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word.

All of them lead to good performance.

Next, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. 

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).  

In [10]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = str(self.data.sentence[index])
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # prepend outside label for [CLS] token
        labels.append("O") # append outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'] * (maxlen - len(tokenized_sentence))
          labels = labels + ["O"] * (maxlen - len(labels))

        # step 4: obtain the attention mask
        attn_mask = [int(tok != '[PAD]') for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

We're using 80% (.8) of the data for training. The remaining goes for testing.

## TODO: Add validation set when the dataset is big enough
I know there should be a third set called a validation set but my dataset is too small.

In [11]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (676, 2)
TRAIN Dataset: (541, 2)
TEST Dataset: (135, 2)


Let's have a look at the first training example:

In [12]:
print(training_set[0]['ids'].shape)
training_set[0]

torch.Size([128])


{'ids': tensor([  101,  5587,  1996,  7975,  1010, 16130,  2092,  1998,  2404,  1996,
          8962,  1999,  1996,  3653, 20192,  3064, 17428,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

Let's verify that the input ids and corresponding targets are correct:

In [13]:
# print the first 30 tokens and corresponding labels
first_n = 30
first_n_ids = training_set[0]["ids"][:first_n]
first_n_targets = training_set[0]["targets"][:first_n]
for token, label in zip(tokenizer.convert_ids_to_tokens(first_n_ids), first_n_targets):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
add         O
the         O
chicken     U-FOOD
,           U-FOOD
stir        O
well        O
and         O
put         O
the         O
pot         O
in          O
the         O
pre         O
##hea       O
##ted       O
oven        O
.           O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


Now, let's define the corresponding PyTorch dataloaders:

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

We use a Bert pretrained for token classification. Make sure to specify the correct `num_labels`.

In [15]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

#### **Training the model**

In [16]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.7564, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [17]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 5])

In [18]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [19]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # DO NOT REMOVE THESE COMMENTS
        # They're helpful for debugging the Rust side of things.
        
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [20]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.8368562459945679
Training loss per 100 training steps: 0.22920259772477172
Training loss epoch: 0.18892703484743834
Training accuracy epoch: 0.812100541718774
Training epoch: 2
Training loss per 100 training steps: 0.02835949696600437
Training loss per 100 training steps: 0.06489566586321534
Training loss epoch: 0.06372421606005553
Training accuracy epoch: 0.8527975903084943


#### **Evaluating the model**

In [21]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [22]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.050965890288352966
Validation Loss: 0.056504620126147265
Validation Accuracy: 0.8595147185678551


In [23]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

        FOOD       0.43      0.33      0.37       243

   micro avg       0.43      0.33      0.37       243
   macro avg       0.43      0.33      0.37       243
weighted avg       0.43      0.33      0.37       243



/home/h/micromamba/envs/bertner/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/h/micromamba/envs/bertner/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


#### **Inference**

The code below does not take into account when predictions of different word pieces that belong to the same word do not match.

In [28]:
sentence = "I will bake a cake with those eggs."

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]
active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels))
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

special_tokens = ('[CLS]', '[SEP]', '[PAD]')
word_level_predictions = []
for pair in wp_preds:
  if pair[0].startswith(" ##") or pair[0] in special_tokens:
    # skip prediction early
    continue
  word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in special_tokens]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

i will bake a cake with those eggs .
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-FOOD', 'O']


In [27]:
model.save_pretrained('trained_v1_safetensors', safe_serialization=True)
tokenizer.save_pretrained('trained_v1_safetensors', safe_serialization=True)

('trained_v1_safetensors/tokenizer_config.json',
 'trained_v1_safetensors/special_tokens_map.json',
 'trained_v1_safetensors/vocab.txt',
 'trained_v1_safetensors/added_tokens.json')

We won't use pipeline. Our pipeline is RUST baby!

In [26]:
# from transformers import pipeline

# pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
# pipe("Today we'll be baking a cake with only the whites of eggs.")